In [20]:
import requests
import json
import pandas as pd

### Por medio de un Json File vamos a manejar la las Key de la API y de Redshift y tambien usuario Redshift.

In [21]:
with open("config.json") as config_file:
    config=json.load(config_file)

In [22]:
config.keys()

dict_keys(['API_KEY', 'Reshift_KEY', 'Redshift_User'])

In [23]:
Api_Key=config["API_KEY"]

In [24]:
Redshift_Key=config["Reshift_KEY"]

In [25]:
Redshift_User=config["Redshift_User"]

In [26]:
#Conexion a la API de Football-API

In [8]:
import http.client

conn = http.client.HTTPSConnection("v3.football.api-sports.io")

headers = {
    'x-rapidapi-host': "v3.football.api-sports.io",
    'x-rapidapi-key': Api_Key
    }

conn.request("GET", "/leagues", headers=headers)

res = conn.getresponse()
data = res.read()

In [9]:
#Cargando toda la data a un Diccionario
leagues_data=json.loads(data)

In [10]:
#La Key "Response" es la que guarda toda la info que nos importa
leagues_data.keys()

dict_keys(['get', 'parameters', 'errors', 'results', 'paging', 'response'])

### Para el entregable la tabla principal es la de Goles sin embargo cree otra tabla que podria servirme para poder tener los valores fe fecha de inicio y fin de temporada cada año.

In [27]:
def search_league(league_name):
    for x in leagues_data["response"]:
        if x["league"]["name"]==league_name:
            id_league=x["league"]["id"]
            league_info=x["league"]
            df_league=x
            break
    return id_league,league_info,df_league

In [28]:
def create_df(league_name):
    df_league=[]
    df=search_league(league_name)[2]
    for x in df["seasons"]:
        registro={"year":x["year"],"start":x["start"],"end":x["end"]}
        df_league.append(registro)
    df_league=pd.DataFrame.from_dict(df_league)
    
    return df_league

### Esta funcion genera un historico de Fechas de inicio y fin por temporada para la liga que yo quiera.

In [29]:
create_df("Liga Profesional Argentina")

,year,start,end
0,2015,2015-02-13,2015-12-07
1,2016,2016-08-26,2017-06-27
2,2017,2017-08-25,2018-05-15
3,2018,2018-08-10,2019-04-07
4,2019,2019-07-27,2020-03-10
5,2020,2020-10-30,2021-01-18
6,2021,2021-02-12,2021-12-12
7,2022,2022-02-10,2022-10-23
8,2023,2023-01-26,2023-07-30


### Vamos a crear una funcion que tome como argumentos, el ID de un equipo y su liga, y el periodo de años que queremos. Con esos valores la funcion va hacer un llamado por año a la API y luego internamente va combinar los diccionarios que hay dentro de cada año para mesclar los datos y obtener un DF que muestre los goles a favor y en contra por periodo de minuto por año para el equipo y liga seleccionado

In [30]:
# Año, Equipo, Minuto , Goles 

In [15]:
import http.client

conn = http.client.HTTPSConnection("v3.football.api-sports.io")

headers = {
    'x-rapidapi-host': "v3.football.api-sports.io",
    'x-rapidapi-key': Api_Key
    }

conn.request("GET", "/teams/statistics?season=2015&team=435&league=128", headers=headers)

res = conn.getresponse()
stats_data = res.read()

In [31]:
def create_df_stats(team_id,fecha_inicio,fecha_fin,league):
    
    conn = http.client.HTTPSConnection("v3.football.api-sports.io")

    headers = {
        'x-rapidapi-host': "v3.football.api-sports.io",
        'x-rapidapi-key': Api_Key
        }
    
    df_final=[]
    
    for x in range(fecha_inicio,fecha_fin+1):
        code="/teams/statistics?season="+str(x)+"&team="+str(team_id)+"&league="+str(league)
        conn.request("GET", code , headers=headers)
        res = conn.getresponse()
        stats_data = res.read()
        teams_df=json.loads(stats_data)
        #Definiendo variables relevantes
        team_name=teams_df["response"]["team"]["name"]
        ##Definiendo Df de Goles a Favor y Goles en Contra en Memoria que no requiere llamada extra de la API
        goals_for=teams_df["response"]["goals"]["for"]["minute"]
        goals_against=teams_df["response"]["goals"]["against"]["minute"]
        
        #El orden de las Keys del diccionario a Favor y en Contra no deberia Cambiar para juntar la data por Minutos.
        #Vamos a chequear si no hubo algun cambio para no poner goles en rangos equivocados de minutos antes de avanzar
        if list(goals_for.keys())==list(goals_against.keys()):
            
            for minute_for in goals_for.keys():
                range_minutes=minute_for
                total_goals=goals_for[minute_for]["total"]
                total_goals_against=goals_against[minute_for]["total"]
                registro={"Year":x,"Team":team_name,"Minutes":range_minutes,"Goals_For":total_goals,"Goals_Against":total_goals_against}
                df_final.append(registro)            
        else:
            print("Los minutos no Matchean")
            
    df_final=pd.DataFrame.from_dict(df_final)
            
    return df_final

In [32]:
df_river=create_df_stats(435,2015,2023,128)

In [33]:
df_river.head()

,Year,Team,Minutes,Goals_For,Goals_Against
0,2015,River Plate,0-15,4.0,3.0
1,2015,River Plate,16-30,8.0,3.0
2,2015,River Plate,31-45,7.0,5.0
3,2015,River Plate,46-60,6.0,9.0
4,2015,River Plate,61-75,8.0,5.0


## Chequeamos los Missing Values del DataFrame

In [34]:
df_river.isnull().sum()

Year              0
Team              0
Minutes           0
Goals_For        11
Goals_Against    10
dtype: int64

### Los missing_values se deben a que hubo ciertos años, y cierta franja de minutos en los que no se metio goles y tampoco se recibio goles. Eso equivale a 0 goles en dicho periodo por lo que se va transformar a 0

In [35]:
df_river=df_river.fillna(0)

### Como ultimo paso antes de cargar la tabla a Redshift vamos a transformar los goles de valor Flotante a Entero, ya que los goles son variable discreta.

In [36]:
df_river["Goals_For"]=df_river["Goals_For"].astype(int)
df_river["Goals_Against"]=df_river["Goals_Against"].astype(int)
df_river["Year"] = pd.to_datetime(df_river["Year"], format="%Y")
df_river["Year"]=df_river["Year"].dt.year

### DataFrame Final

In [37]:
df_river

,Year,Team,Minutes,Goals_For,Goals_Against
0,2015,River Plate,0-15,4,3
1,2015,River Plate,16-30,8,3
2,2015,River Plate,31-45,7,5
3,2015,River Plate,46-60,6,9
4,2015,River Plate,61-75,8,5
...,...,...,...,...,...
67,2023,River Plate,46-60,7,2
68,2023,River Plate,61-75,8,6
69,2023,River Plate,76-90,10,4
70,2023,River Plate,91-105,4,1


## Crear conexion a Redshift

In [38]:
import psycopg2
url="data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com"
data_base="data-engineer-database"
user=Redshift_User


try:  
    conn = psycopg2.connect(
        host='data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com',
        dbname=data_base,
        user=user,
        password=Redshift_Key,
        port='5439')
    print("Conectado a Redshift con éxito!")

        
except Exception as e:
    print("No es posible conectar a Redshift")
    print(e)

Conectado a Redshift con éxito!


In [39]:
# Crear un cursor:
cur = conn.cursor()

# Ejecutar la sentencia DROP TABLE:
cur.execute("DROP TABLE IF EXISTS Goals")

# Hacer commit para aplicar los cambios:
conn.commit()

In [40]:
#Crear la tabla si no existe:
with conn.cursor() as cur:
    cur.execute("""
        CREATE TABLE IF NOT EXISTS Goals (
            Year INT,
            Team VARCHAR(255),
            Minutes VARCHAR(10),
            Goals_For INT,
            Goals_Against INT
        )
    """)
    conn.commit()

In [41]:
#consultando la tabla Goles:
cur = conn.cursor()
cur.execute("SELECT * FROM Goals")
results = cur.fetchall()
#results

In [42]:
from psycopg2.extras import execute_values

with conn.cursor() as cur:
    execute_values(
        cur,
        '''
        INSERT INTO Goals (Year, Team, Minutes, Goals_For, Goals_Against)
        VALUES %s
        ''',
        [tuple(row) for row in df_river.values],
        page_size=len(df_river)
    )
    conn.commit()

In [43]:
# Vamos a revisar la tabla

cur = conn.cursor()
cur.execute("SELECT * FROM Goals")
results = cur.fetchall()

In [44]:
# Veo cómo quedó la tabla "canciones" en Redshift. Convierto "results" al DataFrame "d
column_names=["Year", "Team", "Minutes", "Goals_For", "Goals_Against"]
df_redshift = pd.DataFrame(results, columns=column_names)
df_redshift.head()

,Year,Team,Minutes,Goals_For,Goals_Against
0,2015,River Plate,0-15,4,3
1,2015,River Plate,16-30,8,3
2,2015,River Plate,31-45,7,5
3,2015,River Plate,46-60,6,9
4,2015,River Plate,61-75,8,5


In [45]:
# Cierro tanto el cursor como la conexión a la base de datos:
cur.close()
conn.close()